# Azure AI Agents с поддръжка на Model Context Protocol (MCP)

Този notebook демонстрира как да използвате Azure AI Agents с инструменти на Model Context Protocol (MCP), за да създадете интелигентен агент, който може да използва външни MCP сървъри за разширени възможности.


## Инсталиране на необходимите NuGet пакети

Първо, трябва да инсталираме пакета Azure AI Agents Persistent, който предоставя основната функционалност за работа с Azure AI Agents.


## Предимства на удостоверяване без ключове

Тази тетрадка демонстрира **удостоверяване без ключове**, което предоставя няколко предимства:
- ✅ **Без управление на API ключове** - Използва удостоверяване, базирано на идентичност в Azure
- ✅ **Подобрена сигурност** - Няма съхранени тайни в кода или конфигурацията
- ✅ **Автоматично обновяване на удостоверенията** - Azure управлява жизнения цикъл на удостоверенията
- ✅ **Достъп, базиран на роли** - Използва Azure RBAC за детайлно управление на разрешенията

`DefaultAzureCredential` автоматично използва най-добрия наличен източник на удостоверения:
1. Управлявана идентичност (когато се изпълнява в Azure)
2. Удостоверения от Azure CLI (по време на разработка)
3. Удостоверения от Visual Studio
4. Променливи на средата (ако са конфигурирани)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Инсталирайте пакета Azure Identity за удостоверяване с услугите на Azure, използвайки DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Импортиране на необходимите пространства от имена

Импортирайте необходимите пространства от имена за Azure AI Agents и Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Конфигуриране на Azure AI Agent Client (Автентикация без ключ)

Настройте конфигурационните променливи и създайте PersistentAgentsClient, използвайки **автентикация без ключ**:
- **projectEndpoint**: Крайна точка на проекта в Azure AI Foundry
- **modelDeploymentName**: Името на внедрения AI модел (GPT-4.1 nano)
- **mcpServerUrl**: URL адрес на MCP сървъра (Microsoft Learn API)
- **mcpServerLabel**: Етикет за идентифициране на MCP сървъра
- **DefaultAzureCredential**: Използва управлявана идентичност, Azure CLI или други източници на идентификация (не са необходими API ключове)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Създаване на дефиниция за MCP инструмент

Създайте дефиниция за MCP инструмент, който се свързва със сървъра Microsoft Learn MCP. Това ще позволи на агента да има достъп до съдържание и документация от Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Създайте AI агент

Създайте постоянен AI агент със зададения модел и MCP инструменти. Агентът е конфигуриран със следното:
- Модел GPT-4.1 nano
- Инструкции за използване на MCP инструменти за помощ
- Достъп до сървъра Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Създаване на нишка и изпращане на съобщение

Създайте нишка за разговор и изпратете съобщение до потребителя с въпрос за разликата между Azure OpenAI и OpenAI. Това ще тества способността на агента да използва MCP инструментите за предоставяне на точна информация.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Конфигуриране на ресурси за MCP инструмент (Без ключ)

Настройте ресурсите за MCP инструмента. За напълно безключов подход можете да премахнете персонализираните хедъри, ако MCP сървърът поддържа удостоверяване, базирано на Azure идентичност. Примерът по-долу показва как да добавите хедъри, ако е необходимо, но за безключови сценарии те може да не са нужни.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Стартиране на изпълнение на агент

Създайте и стартирайте изпълнение за обработка на съобщението на потребителя. Агентът ще използва конфигурираните MCP инструменти и ресурси, за да генерира отговор.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Наблюдение на изпълнението и управление на одобренията за инструменти (без ключ)

Следете статуса на изпълнението на агента и управлявайте необходимите одобрения за инструменти. Този цикъл:
1. Изчаква изпълнението да завърши или да изисква действие
2. Автоматично одобрява извикванията на инструменти MCP, когато е необходимо
3. За удостоверяване без ключ, заглавията може да не са нужни, ако сървърът MCP поддържа Azure идентичност


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Показване на резултати от разговора

Извлечете и покажете всички съобщения в нишката, като представите както въпроса на потребителя, така и отговора на агента. Съобщенията се показват в хронологичен ред с времеви отметки и индикатори за ролите.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
